# Introduction to Smartnoise-SQL

[Smartnoise-SQL](https://docs.smartnoise.org/sql/index.html) is a python library that enables to perform differentially private SQL queries. 

SmartNoise is intended for scenarios where the analyst is trusted by the data owner.

## Step 1: Install the Library

Smartnoise-sql is available on pypi, it can be installed via the pip command. We will use the latest version of the library to date: version 1.0.6.

In [1]:
!pip install smartnoise-sql==1.0.6

## Step 2: Load and Prepare Data

In this notebook, we will work with the [penguin dataset]("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv") from [seaborn datasets](https://github.com/mwaskom/seaborn-data).
We load the dataset via pandas in a dataframe `df`.

In [2]:
import pandas as pd

In [3]:
path_to_data = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv"
df = pd.read_csv(path_to_data)

We can look at the first rows of the dataframe to get to know the data:

In [4]:
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


We see that there are 7 columns: 'species', 'island', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g' and 'sex' with various data types.

## Step 3: Prepare Analysis with Smartnoise-SQL

Before doing a query, `smartnoise-sql` requires a reader object [(see doc here)](https://docs.smartnoise.org/sql/api/index.html#snsql.connect.from_df). When working with pandas dataframe, this object takes three parameters:
- df: The Pandas DataFrame to be queried (which we loaded in step 2)
- privacy: A Privacy object with the desired privacy parameters (we instantiate it in the next subsection)
- metadata: The metadata describing the raw data (we instantiate it afterwards)

### Initialise the Privacy Object

The [`Privacy`](https://docs.smartnoise.org/sql/api/index.html#privacy) object is used to configure the privacy budget for queries. The budget is defined by the parameters $\epsilon$ and $\delta$, following the standard definition of *approximate differential privacy*:  

$$
Pr[M(S) \in O] \leq e^{\epsilon} \cdot Pr[M(S') \in O] + \delta
$$  

In this example, we set $\epsilon = 0.1$ and $\delta = 0.0001$.


**Note**: Each query in `smartnoise-sql` is internally decomposed into one or more sub-queries. Differential privacy noise is applied to each sub-query, and the results are then combined to produce the final answer.  

The budget you specify applies per sub-query, not per overall query.  

For example, an `AVG` query is computed as:  

- one sub-query for the sum 
- one sub-query for the count

Each sub-query consumes the full $(\epsilon, \delta)$ budget. The private mean is then returned as:  

$$
dp\_mean = \frac{dp\_sum}{dp\_count}
$$  

Thus, the total cost of the query is approximately double the specified budget.  
See the advanced exercise in step 6 for a detailed example of this mechanism.

In [5]:
from snsql import Privacy

In [6]:
# TODO: fill epsilon and delta values
# EPSILON = ...
# DETLA = ...

# Correction
EPSILON = 0.1
DELTA = 1/10000

In [7]:
privacy = Privacy(epsilon=EPSILON, delta=DELTA)

### Prepare the metadata

Next we prepare the metadata. The expected format is described in the 
[smartnoise-sql documentation](https://docs.smartnoise.org/sql/metadata.html#metadata).  

Metadata can be provided in different formats, such as an external `yaml` file or a dictionary.  
In this notebook we will use the [dictionary format](https://docs.smartnoise.org/sql/metadata.html#dictionary-format).

There are two categories of options: Table Options and Column Options.


#### Table Options
These apply to the entire table and control general query behavior.  
They have default values and should only be overridden with care.

- `max_ids`: maximum number of rows in which a unique user can appear.  
  In our case, each penguin appears only once, so `max_ids = 1`.  
- `row_privacy`: specifies whether each row corresponds to a single individual.  
  This is true for our dataset since each row is one penguin.  
- `censor_dims`: drops `GROUP BY` output rows that could reveal the presence of individuals.
  For instance, if a categorical value is not public information and unique to an individual, its presence in the result can be disclosive.
  Since species, island, and sex are public information, we set this to `false`.  
- All other fields are kept at their default values.  


#### Column Options
These are required and must describe the table column by column.

- Each column name must exactly match the column name in the table.  
- Each column must define a `type` field, which can be one of: `int`, `float`, `string`, `boolean`, or `datetime`.  
- Numeric columns (`int` or `float`) should also define `lower` and `upper` bounds, representing the theoretical minimum and maximum values.  
  These bounds are used to compute sensitivity and calibrate the noise for differential privacy.  
- Optionally, a boolean `nullable` field can be provided.  
  By default it is `true`, meaning columns may contain null values.  
  If the user knows a column has no nulls, this can be set to `false`.  
- Other options exist but are not covered in this notebook.  


As explained for `Table Options`, we specify `max_ids`, `row_privacy` and `censor_dims` (and keep all other the default values too).

In [8]:
table_options = {
    'max_ids': 1,
    'row_privacy': True,
    'censor_dims': False
}

For `Column Options`, we look at the list of columns and dtypes and use expert knowledge to determine the type and bounds:

In [9]:
df.dtypes

species               object
island                object
bill_length_mm       float64
bill_depth_mm        float64
flipper_length_mm    float64
body_mass_g          float64
sex                   object
dtype: object

In [10]:
# Specify metadata for each column
species_col = {'type': 'string', 'nullable': False}  # string: means chain of characters (like text)
island_col = {'type': 'string', 'nullable': False}
bill_length_col = {'type': 'float', 'lower': 30.0, 'upper': 65.0}  # float: means a decimal number (like 3.14)
bill_depth_col = {'type': 'float', 'lower': 13.0, 'upper': 23.0}
flipper_length_col = {'type': 'float', 'lower': 150.0, 'upper': 250.0}

In [11]:
# TODO: Fill body_mass_g and sex column metadata knowing that these specied of penguins typically weight between 2000.0 and 7000.0 grammes.
# We cannot say for sure that there are no nulls in these columns.
# body_mass_g_col = ...
# sex_col = ...

# Correction
body_mass_g_col = {'type': 'float', 'lower': 2000.0, 'upper': 7000.0}
sex_col = {'type': 'string'}

In [12]:
# Collect all metadata into "columns_options"
columns_options = {
    'species': species_col,
    'island': island_col,
    'bill_length_mm': bill_length_col,
    'bill_depth_mm': bill_depth_col,
    'flipper_length_mm': flipper_length_col,
    'body_mass_g': body_mass_g_col,
    'sex': sex_col, 
}

Then, we assemble the metadata respecting the format expected by `smartnoise-sql`. As it can work on multiple table, it needs some additional outer fields.

In [13]:
# Build metadata
metadata = {
    '': {
        '': {
            'penguin_table': {  # Note: we set the table name as 'penguin_table' here
                **table_options,
                **columns_options
            }
        }
    }
}

In [14]:
metadata

{'': {'': {'penguin_table': {'max_ids': 1,
    'row_privacy': True,
    'censor_dims': False,
    'species': {'type': 'string', 'nullable': False},
    'island': {'type': 'string', 'nullable': False},
    'bill_length_mm': {'type': 'float', 'lower': 30.0, 'upper': 65.0},
    'bill_depth_mm': {'type': 'float', 'lower': 13.0, 'upper': 23.0},
    'flipper_length_mm': {'type': 'float', 'lower': 150.0, 'upper': 250.0},
    'body_mass_g': {'type': 'float', 'lower': 2000.0, 'upper': 7000.0},
    'sex': {'type': 'string'}}}}}

### Instantiate the reader

All arguments are now available to create the reader object mentionned at the begin of step 3.

In [15]:
from snsql import from_df

In [16]:
# TODO: Instantiate the reader object
# reader = ...

# Correction
reader = from_df(df, metadata=metadata, privacy=privacy)

## Step 4: Differentially Private Dataset Query

We are now ready to execute queries. Note: the table was named `penguin_table` in the metadata, so the queries must always query `FROM penguin_table`.

### Count Number of Penguins
Let's write a query to count the number of rows in the dataset:

In [17]:
COUNT_QUERY = "SELECT COUNT(*) AS nb_penguins FROM penguin_table"

We can verify the privacy cost of this query:

In [18]:
epsilon, delta = reader.get_privacy_cost(COUNT_QUERY)
print(f"This query will cost {epsilon} epsilon and {delta} delta.")

This query will cost 0.1 epsilon and 0.00014999500000001387 delta.


We can request its accuracy for a given $\alpha$, i.e., the interval within which the unprotected query has a probability $1- \alpha$ to fall within. For instance, with $\alpha=0.05$ (95\% interval):

In [19]:
ALPHA = 0.05
accuracy = reader.get_simple_accuracy(COUNT_QUERY, ALPHA)
print(f"For 95% of query executions, the number of penguins will be within +/- {accuracy[0]} of true value")

For 95% of query executions, the number of penguins will be within +/- 29.95732301453925 of true value


And then run the query to obtain the protected result:

In [20]:
result = reader.execute(COUNT_QUERY)
result

[['nb_penguins'], [336]]

And compare it with the true number of rows:

In [21]:
print(f"Non-DP number of penguins: {df.shape[0]}.")
print(f"DP number of penguins: {result[1][0]}.")
print(f"Difference: {abs(df.shape[0] - result[1][0])}.")

Non-DP number of penguins: 344.
DP number of penguins: 336.
Difference: 8.


### Average Bill Length of Penguins
Beyond counting queries, we now write and experiment with averages of the bill length.

In [22]:
privacy = Privacy(epsilon=EPSILON, delta=DELTA)
reader = from_df(df, metadata=metadata, privacy=privacy)

In [23]:
# TODO: write a query to compute the average bill length, check its budget and execute it
# AVG_QUERY = ...
# epsilon, delta = ...
# result = ...

# Correction
AVG_QUERY = "SELECT AVG(bill_length_mm) AS avg_bill_length_mm FROM penguin_table"
epsilon, delta = reader.get_privacy_cost(AVG_QUERY)
result = reader.execute(AVG_QUERY)

print(f"This query will cost {epsilon} epsilon and {delta} delta.")
print(f"DP average bill length: {result[1][0]}mm.")

This query will cost 0.2 epsilon and 4.999999999999449e-05 delta.
DP average bill length: 50.25418280164734mm.


### Overriding the mechanism
SmartNoiseSQL uses default protection mechanism depending of the type of the query. For the average, SmanrtNoiseSQL applies a Laplace mechanism by default, but we would like to use Gaussian differential privacy, which uses the Gaussian mechanism instead. This can be done by overriding the mechanism in the `Privacy` object.  

When computing an average, both the sum and the count are computed as sub-queries, so the mechanism needs to be overridden for each of them.  

In [24]:
from snsql import Stat, Mechanism

In [25]:
print(f"By default, smartnoise-sql default to using {privacy.mechanisms.map[Stat.count]} for counts.")
print(f"By default, smartnoise-sql default to using {privacy.mechanisms.map[Stat.sum_float]} for sums of float.")

print("We switch to use Gaussian mechanisms")
privacy.mechanisms.map[Stat.count] = Mechanism.gaussian
privacy.mechanisms.map[Stat.sum_float] = Mechanism.gaussian

By default, smartnoise-sql default to using Mechanism.discrete_laplace for counts.
By default, smartnoise-sql default to using Mechanism.laplace for sums of float.
We switch to use Gaussian mechanisms


Estimating the privacy cost of the query again, we see that a much bigger $\delta$ is spent with the Gaussian mechanism.

In [26]:
AVG_QUERY = "SELECT AVG(bill_length_mm) AS avg_bill_length_mm FROM penguin_table"
epsilon, delta = reader.get_privacy_cost(AVG_QUERY)
print(f"This query will cost {epsilon} epsilon and {delta} delta.")

This query will cost 0.2 epsilon and 0.0002499800004999164 delta.


Technical Note: even though we where using the Laplace mechnism previously, the queries were consuming some $\delta$. The reason behind this surprising behaviour is that SmartNoiseSQL uses an improved composition theorem for which budget expenditure for an increasing number of queries is reduced at the cost of a small $\delta$.

### Average Bill Length by Species of Penguins

We now reset the privacy and reader to default parameters (with Laplace mechanisms) by re-creating the objects :

In [27]:
privacy = Privacy(epsilon=EPSILON, delta=DELTA)
reader = from_df(df, metadata=metadata, privacy=privacy)

Now write a query to compute the average bill length per species to get a deeper understanding of the population characteristics:

In [28]:
# TODO: write a query to compute the average bill length of penguin per species
#AVG_GROUP_QUERY = """
#    SELECT 
#    species AS species, \
#    ... AS avg_bill_length_mm \
#    FROM ... \
#    GROUP BY ...
#"""

# Correction
AVG_GROUP_QUERY = """
    SELECT 
    species AS species,
    AVG(bill_length_mm) AS avg_bill_length_mm
    FROM penguin_table
    GROUP BY species
"""

And verify the privacy budget of the query. Note: can you guess what it is before executing the cell ?

In [29]:
epsilon, delta = reader.get_privacy_cost(AVG_GROUP_QUERY)
print(f"This query will cost {epsilon} epsilon and {delta} delta.")

This query will cost 0.2 epsilon and 4.999999999999449e-05 delta.


Now, we execute it:

In [30]:
avg_result = reader.execute(AVG_GROUP_QUERY)

And just format the result:

In [31]:
def format_results(results):
    """Small function to format results in pd.DataFrame"""
    result = results[:]
    cols = result.pop(0)
    return pd.DataFrame(result, columns=cols)

In [32]:
avg_result = format_results(avg_result)
avg_result

,species,avg_bill_length_mm
0,Adelie,33.213546
1,Chinstrap,45.581348
2,Gentoo,46.131811


## Step 5: Learn How to Use the Odometer

When working with differentially private queries, there are two common approaches to tracking privacy loss budget spending.

1. **Summing the privacy cost of each individual query manually** – i.e., calculating epsilon and delta for one query and adding it for each executions.  
2. **Using the privacy odometer** – a mechanism provided by the `PrivateReader` that tracks accumulated privacy spending automatically as queries are executed.

We will show the two versions here. First, we initialise the reader and check that the privacy spent so far is $0$.

In [33]:
privacy = Privacy(epsilon=0.1, delta=10e-7)
reader = from_df(df, metadata=metadata, privacy=privacy)
reader.odometer.spent

(0.0, 0.0)

We execute one query and check the cost of a single query with `get_privacy_cost` and with the odometer.

In [34]:
# Execute one query
query = 'SELECT AVG(bill_length_mm), AVG(flipper_length_mm) FROM penguin_table'
result = reader.execute(query)

# Privacy loss budget of one query with the odometer
epsilon_odometer, delta_odometer = reader.odometer.spent
print(f"Privacy loss budget of one query with the odometer ({epsilon_odometer}, {delta_odometer}).")

# Privacy loss budget of one query
epsilon_single, delta_single = reader.get_privacy_cost(query)
print(f"Privacy loss budget of one query (no odometer) ({epsilon_single}, {delta_single}).")

Privacy loss budget of one query with the odometer (0.4, 4.999999999588667e-07).
Privacy loss budget of one query (no odometer) (0.4, 4.999999999588667e-07).


We see that they are the same. Now let's compare the privacy loss budget of a hundred queries:

In [35]:
for _ in range(99):
    reader.execute(query)

epsilon_odometer_many, delta_odometer_many = reader.odometer.spent
print(f"Privacy loss budget of 100 queries with the odometer ({epsilon_odometer_many}, {delta_odometer_many}).")
print(f"Privacy loss budget of 100 queries (no odometer) ({epsilon_single * 100}, {delta_single * 100}).")

Privacy loss budget of 100 queries with the odometer (12.771879536126042, 4.999999999588667e-07).
Privacy loss budget of 100 queries (no odometer) (40.0, 4.999999999588667e-05).


Question: When doing multiple queries, is it better to use the odometer or is it better to simply add the cost of each individual query ? Why ?

**Short Answer**: The odometer because less budget is spent.

**Long Answer**: While multiplying the cost of a single query by the number of executions may seem straightforward, it assumes a worst-case additive composition of privacy. This approach can significantly overstate the true privacy loss, especially when queries are correlated or when advanced composition theorems apply.  

The odometer, on the other hand, tracks the actual accumulated privacy spending using the most advantageous composition theorem. This means it can leverage advanced composition, parallel composition, and other optimizations built into the SmartNoise framework. 

In short, the odometer is better than simply adding up individual query costs because it optimizes privacy composition of multiple queries and provides a tighter, more accurate estimate of total privacy consumption.  

## Step 6: Advanced Exercise with Hypothesis testing

We want to test if bill length differ between two penguins species with a two-tailed two-sample $t$ test and a level of significance at 0.05.
- The null hypothese $H_0$ is bill length does not differ between species.
- The alternative hypothesis $H_a$ is bill length does differ between species.

Therefore, we need 
- the number of penguin per species,
- the average bill length per species and,
- the standard deviation of bill length per species.

We first allow more budget for this query a create new the `privacy` and `reader` objects.

In [36]:
privacy = Privacy(epsilon=1.0, delta=0.1)
reader = from_df(df, metadata=metadata, privacy=privacy)

### Write query

Now write a query to get the necessary statistics:

In [37]:
# TODO: Write a query that gets the number of penguin, average and standard deviation of bill length per penguin species
#HYP_QUERY = """
#    SELECT
#    species AS species,
#    ... AS nb_penguin,
#    ... AS avg_bill_length_mm,
#    ... AS std_bill_length_mm
#    FROM ...
#    GROUP BY ...
#"""

# Correction
HYP_QUERY = """
    SELECT
    species AS species,
    COUNT(bill_length_mm) AS nb_penguin,
    AVG(bill_length_mm) AS avg_bill_length_mm,
    STD(bill_length_mm) AS std_bill_length_mm
    FROM penguin_table
    GROUP BY species
"""

### Understand budget

Do you remember the command to estimate the budget of this query?

In [38]:
# TODO: Estimate budget of query with smartnoise_sql
#epsilon, delta = ...

# Correction
epsilon, delta = reader.get_privacy_cost(HYP_QUERY)
epsilon, delta

(np.float64(3.0), np.float64(0.050000000000000044))

We see that the privacy loss budget spent in epsilon is three times the input budget epsilon. Let's look into into smartnoise-sql to understand why (this is not necessary to use the library but unables us to understand how it works:

In [39]:
subquery, query = reader._rewrite(HYP_QUERY)
print("Outer query:", query)
print("")
print("Subquery:", subquery)

Outer query: SELECT species AS species, ( count_bill_length_mm ) AS nb_penguin, ( ( sum_bill_length_mm / count_bill_length_mm ) ) AS avg_bill_length_mm, ( SQRT ( ( sum_alias_0x4ec5 / count_bill_length_mm ) - ( sum_bill_length_mm / count_bill_length_mm ) * ( sum_bill_length_mm / count_bill_length_mm ) ) ) AS std_bill_length_mm FROM ( SELECT species AS species, COUNT(bill_length_mm) AS count_bill_length_mm, SUM(bill_length_mm) AS sum_bill_length_mm, SUM(bill_length_mm * bill_length_mm) AS sum_alias_0x4ec5 FROM ( SELECT species AS species, CASE WHEN bill_length_mm < 30.0 THEN 30.0 WHEN bill_length_mm > 65.0 THEN 65.0 ELSE  bill_length_mm END AS bill_length_mm FROM penguin_table ) AS per_key_all GROUP BY species ) AS exact_aggregates

Subquery: SELECT species AS species, COUNT(bill_length_mm) AS count_bill_length_mm, SUM(bill_length_mm) AS sum_bill_length_mm, SUM(bill_length_mm * bill_length_mm) AS sum_alias_0x4ec5 FROM ( SELECT species AS species, CASE WHEN bill_length_mm < 30.0 THEN 30.0

What happened is that smartnoise sql rewrote the query as three subqueries (without counting the SELECT on species):
- SELECT species AS species,
- COUNT(bill_length_mm) AS count_bill_length_mm,
- SUM(bill_length_mm) AS sum_bill_length_mm,
- SUM(bill_length_mm * bill_length_mm) AS sum_alias_0xc15f

And then postprocessed them with:
- SELECT species AS species,
- ( count_bill_length_mm ) AS nb_penguin,
- ( ( sum_bill_length_mm / count_bill_length_mm ) ) AS avg_bill_length_mm,
- ( SQRT ( ( sum_alias_0xc15f / count_bill_length_mm ) - ( sum_bill_length_mm / count_bill_length_mm ) * ( sum_bill_length_mm / count_bill_length_mm ) ) ) AS std_bill_length_mm

The get the result of the query. 

Each subquery cost the epsilon given in input. Therefore, the total is three times the input budget.

### Execute query

We execute the query below. If there is a `nan` in the output, we execute the query again.

In [40]:
hyp_result = reader.execute(HYP_QUERY)
df_result = format_results(hyp_result)

while df_result.isna().any().any():
    hyp_result = reader.execute(HYP_QUERY)
    df_result = format_results(hyp_result)
df_result

/opt/python/lib/python3.13/site-packages/snsql/_ast/expressions/numeric.py:144: RuntimeWarning: invalid value encountered in sqrt
  return funcs[self.name.lower()](exp)
/opt/python/lib/python3.13/site-packages/snsql/_ast/expressions/numeric.py:144: RuntimeWarning: invalid value encountered in sqrt
  return funcs[self.name.lower()](exp)


,species,nb_penguin,avg_bill_length_mm,std_bill_length_mm
0,Adelie,151,39.004267,7.363224
1,Chinstrap,68,48.698734,10.975031
2,Gentoo,122,46.584159,8.063875


### Testing mean differences between pairs of groups

We now apply a t-test to assess if there is a significance difference in mean bill length between pairs of groups:

In [41]:
import numpy as np

In [42]:
# Create function allowing to compute the test statistic
def t_test(avg_1, avg_2, std_1, std_2, nb_1, nb_2):
    standard_error = (std_1 * (nb_1 - 1) + std_2 * (nb_2 - 1))/(nb_1 + nb_2 - 2)
    return (avg_1 - avg_2)/np.sqrt(standard_error**2*(1/nb_1 + 1 /nb_2))

In [43]:
# Store each individual statistic into named variables
nb_0, avg_0, std_0 = df_result[['nb_penguin', 'avg_bill_length_mm', 'std_bill_length_mm']].iloc[0]
nb_1, avg_1, std_1 = df_result[['nb_penguin', 'avg_bill_length_mm', 'std_bill_length_mm']].iloc[1]
nb_2, avg_2, std_2 = df_result[['nb_penguin', 'avg_bill_length_mm', 'std_bill_length_mm']].iloc[2]

In [44]:
# Compute the statistics for all pairs
t_01 = t_test(avg_0, avg_1, std_0, std_1, nb_0, nb_1)
t_02 = t_test(avg_0, avg_2, std_0, std_2, nb_0, nb_2)
t_12 = t_test(avg_1, avg_2, std_1, std_2, nb_1, nb_2)

CRITICAL_VALUE = 0.05
print(f"T test between species 0 and specie 1: {np.round(t_01, 2)}. Reject null hypothesis: {abs(t_01) > CRITICAL_VALUE}.")
print(f"T test between species 0 and specie 2: {np.round(t_02, 2)}. Reject null hypothesis: {abs(t_02) > CRITICAL_VALUE}.")
print(f"T test between species 1 and specie 2: {np.round(t_12, 2)}. Reject null hypothesis: {abs(t_12) > CRITICAL_VALUE}.")

T test between species 0 and specie 1: -7.83. Reject null hypothesis: True.
T test between species 0 and specie 2: -8.11. Reject null hypothesis: True.
T test between species 1 and specie 2: 1.54. Reject null hypothesis: True.


Technical note 1 : here, we for simplicity we used the same confidence level $\alpha = 0.05$ for all tests. In practice, we should account for multiple testing by at least applying a Bonferroni correction of any other, more advanced, correction.

Technical note 2 : the noise added through the DP mechasnism influences the test performance. Thus formal analysis of the impact of the DP mechnanism on the power of the hypothesis test is necessary. We will discuss this topic on Day 3 of the course.

We also compute the confidence interval of bill length of each species:

In [45]:
ZSCORE = 1.96
df_result['standard_error'] = df_result['std_bill_length_mm']/np.sqrt(df_result['nb_penguin'])
df_result['ci_95_lower_bound'] = df_result['avg_bill_length_mm'] - ZSCORE * df_result['standard_error']
df_result['ci_95_upper_bound'] = df_result['avg_bill_length_mm'] + ZSCORE * df_result['standard_error']
df_result

,species,nb_penguin,avg_bill_length_mm,std_bill_length_mm,standard_error,ci_95_lower_bound,ci_95_upper_bound
0,Adelie,151,39.004267,7.363224,0.599211,37.829814,40.178720
1,Chinstrap,68,48.698734,10.975031,1.330918,46.090135,51.307333
2,Gentoo,122,46.584159,8.063875,0.730069,45.153224,48.015095


## Step 7: On your Own

There are many additional options and methods available in the `smartnoise-sql` library that we could not cover in this course. The key takeaways are:

- **Direct database connections**: `smartnoise-sql` can connect to and execute queries directly on databases such as SQL Server, Postgres, Spark, or Presto. In this notebook, we focused on the data science aspects and demonstrated examples only with Pandas DataFrames.  
- **Table metadata**: A solid understanding of [Table metadata](https://docs.smartnoise.org/sql/metadata.html#table-options) is essential, especially if you plan to adjust the default values. We strongly recommend becoming familiar with these settings before making modifications.  
- **Working with multiple tables**: It is possible to query multiple tables and join them together, with the privacy budget shared across all tables involved.  

We encourage you to explore further on your own!